### Emma Klemets, 260775167   
All my work, code, output and written answers are in this notebook.

In [1]:
#for interactive plots - not what I want

# %matplotlib
# %matplotlib inline
# %matplotlib notebook

In [114]:
import numpy as np
import matplotlib

#for pop out plots - windows
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import time
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

Make a 3-D nbody code that calculates the forces by computing the potential, where the potential is found by convolving the density with the (softened) potential from a single particle. The acceleration is then found by taking the gradient of the potential. You will probably wish to use a leapfrog solver with fixed timestep.

In [ ]:
"""
calculates the forces by computing the potential

potential is found by convolving the density with the (softened) potential from a single particle. 
phi = rho x V_1

The acceleration is then found by taking the gradient of the potential. 
acc = grad phi


calculate the potential using the Green's function!
So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
Since (taking 4*pi*G=1): laplacian(phi) = rho ,

solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian

- how does it not matter which particle for rho?
- what is the r in G's relative to - should it be between two particles? 
"""


In [106]:
class Particles:
    def __init__(self,x,v,m):
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
        self.f=np.empty(x.shape)
        self.a=np.empty(x.shape)
        self.n=self.x.shape[0]
        
    def r(self):
        return np.sqrt(np.sum(self.x**2, axis=1))
        
    def get_forces(self, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3
        
    #this is for calculting forces with position values (xx) that are not ever saved
    def get_forces_2(self, xx, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-xx[i,:]+xx[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3

    def get_acc(self, soft=0.01, do_pot=False):
        self.a[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.a[i,:]=self.a[i,:]+dx*r3 #going to be the same really as F as m=1 rn

    
    def take_step_simple(self,dt): #this is bad, but it does what it should - no dt softening here
        self.get_forces()
        self.x=self.x+dt*self.v
        self.v=self.v+dt*self.f
        
    def take_step_better(self, dt, soft=0.01): #works, is decent - also no dt softening
        xx=self.x+0.5*self.v*dt #first half step
        #gets forces for centerpoint of step
        self.get_forces_2(xx, soft)
        f=self.f
        
        #gets velocity for centerpoint of step
        vv=self.v+0.5*dt*f
        self.x=self.x+dt*vv
        self.v=self.v+dt*f

    def take_step_leapfrog(self, dt, soft=0.01):
        self.get_acc()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.v = v_h #so the V saved will be a half step behind x
        

In [111]:
n=2
#for random starting
# x=np.random.randn(n,2)
# v=np.random.randn(n,2)

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=1
x[1,0]=-1
x[0,2]=1
x[1,2]=-1
v[0,1]=0.25
v[1,1]=-0.25

# x[2,0]=-0.5
# v[2,1]=0

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig, ax = plt.subplots()      

# for k in range(10):
#     for l in range(10):
        
for k in range(100):
    for l in range(20):
#         parts.take_step_simple(dt)
#         parts.take_step_better(dt)
        parts.take_step_leapfrog(dt)
    
    print('step ',k)
#     print(parts.x)
#     f,pot=get_forces(x,do_pot=True)
#     kin=np.sum(v**2)/2
    #print(kin,pot,kin-pot/2)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], color=colors[n_i])
#     plt.axis([-2,2,-2,2])
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)

print()

step  0
step  1
step  2
step  3
step  4
step  5
step  6
step  7
step  8
step  9
step  10
step  11
step  12
step  13
step  14
step  15
step  16
step  17
step  18
step  19
step  20
step  21
step  22
step  23
step  24
step  25
step  26
step  27
step  28
step  29
step  30
step  31
step  32
step  33
step  34
step  35
step  36
step  37


KeyboardInterrupt: 

step  38
step  39
step  40
step  41
step  42
step  43
step  44
step  45
step  46
step  47
step  48


KeyboardInterrupt: 

step  49


KeyboardInterrupt: 

step  50
step  51
step  52
step  53
step  54
step  55
step  56
step  57
step  58
step  59
step  60
step  61
step  62
step  63
step  64
step  65
step  66
step  67
step  68
step  69
step  70
step  71
step  72
step  73
step  74
step  75
step  76
step  77
step  78
step  79
step  80
step  81
step  82
step  83
step  84
step  85
step  86
step  87
step  88
step  89
step  90
step  91
step  92
step  93
step  94
step  95
step  96
step  97
step  98
step  99



In [129]:
x = np.array([[3, 4, 0], [3, 4, 1]])

np.sqrt(np.sum(x**2, axis=1))

array([5.        , 5.09901951])

### Part 1: 
Using this code, show that a single particle starting at rest remains motionless. 

In [118]:
n=1
#single particle at rest
x=np.random.randn(n,3)
v=np.random.randn(n,3)*0

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig = plt.figure()
ax = Axes3D(fig)     

for k in range(1000):
    for l in range(20):
        parts.take_step_leapfrog(dt)
    
#     print('step ',k)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
        
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)


KeyboardInterrupt: 

### Part 2: 
Next, show that a pair of particles placed in a circular orbit continue to orbit each other, for at least some reasonable length of time. 

In [117]:
n=2
#two particles in circular orbit

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=1
x[1,0]=-1
x[0,2]=1
x[1,2]=-1

v[0,1]=0.25
v[1,1]=-0.25
v[0,2]=0.25
v[1,2]=-0.25

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
        
for k in range(300):
    for l in range(20):
        parts.take_step_leapfrog(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)

step  0
step  1
step  2
step  3
step  4
step  5
step  6
step  7
step  8
step  9
step  10
step  11
step  12
step  13
step  14
step  15
step  16
step  17
step  18
step  19
step  20
step  21
step  22
step  23
step  24
step  25
step  26
step  27
step  28
step  29
step  30
step  31
step  32
step  33
step  34
step  35
step  36
step  37
step  38
step  39
step  40
step  41
step  42
step  43
step  44
step  45
step  46
step  47
step  48
step  49
step  50
step  51
step  52
step  53
step  54
step  55
step  56
step  57
step  58
step  59
step  60
step  61
step  62
step  63
step  64
step  65
step  66
step  67
step  68
step  69
step  70
step  71
step  72
step  73
step  74
step  75
step  76
step  77
step  78
step  79
step  80
step  81
step  82
step  83
step  84
step  85
step  86
step  87
step  88
step  89
step  90
step  91
step  92
step  93
step  94
step  95
step  96
step  97
step  98
step  99
step  100
step  101
step  102
step  103
step  104
step  105
step  106
step  107
step  108
step  109
step  110


KeyboardInterrupt: 

step  125


KeyboardInterrupt: 

step  126
step  127
step  128
step  129
step  130
step  131
step  132
step  133
step  134
step  135
step  136
step  137
step  138
step  139
step  140
step  141
step  142
step  143
step  144
step  145
step  146
step  147
step  148
step  149
step  150
step  151
step  152
step  153
step  154
step  155
step  156
step  157
step  158
step  159
step  160
step  161
step  162
step  163
step  164
step  165
step  166
step  167
step  168
step  169
step  170
step  171
step  172
step  173
step  174
step  175
step  176
step  177
step  178
step  179
step  180
step  181
step  182
step  183
step  184
step  185
step  186
step  187
step  188
step  189
step  190
step  191
step  192
step  193
step  194
step  195
step  196
step  197
step  198
step  199
step  200
step  201
step  202
step  203
step  204
step  205
step  206
step  207
step  208
step  209
step  210
step  211
step  212
step  213
step  214
step  215
step  216
step  217
step  218
step  219
step  220
step  221
step  222
step  223
step  224
step  225


### Part 3: 
Set up both periodic and non-periodic boundary conditions. Set up a problem where hundreds of thousands of particles are initially scattered randomly throughout the domain. Show the evolution with time for both periodic and non-periodic boundary conditions. Track the total energy how well is it conserved? 

### Part 4: 
In cosmology, we start the universe with a scale-invariant power spectrum, so mass fluctuations are proportional to k−3. Start with the particles on a grid, but with masses derived from a realization of k−3 and use periodic boundary conditions (although not strictly necessary, you may with to start with your particles in the center of grid cells rather than at the corners). How does your universe look now?